# Recommendations Systems

<span></span>

### Import Preliminaries

In [9]:
# Import generic data science packages
%matplotlib inline
%config InlineBackend.figure_format='retina'


# Import modules
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import numpy as np
import pandas as pd 
import sklearn
import seaborn as sns
import warnings

# Import Model Selection 
from sklearn.model_selection import train_test_split, cross_val_score

# Set pandas options
pd.set_option('max_columns',1000)
pd.set_option('max_rows',30)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set plotting options
mpl.rcParams['figure.figsize'] = (8.0, 7.0)

# Ignore Warnings
warnings.filterwarnings('ignore')

### Import Data

In [11]:
df = pd.read_csv('/Users/kavi/Documents/Science/Brainstation/Python Labs/Recommendation Systems/movies_metadata.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

### 

### Data Overview

In [8]:
pd.Series(df.adult.value_counts())

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

### Top Rated System

In [19]:
top_df = df.sort_values(by='vote_average', ascending=False)
top_df[['title','vote_average','vote_count']].head(5)

,title,vote_average,vote_count
21642,Ice Age Columbus: Who Were the First Americans?,10.000,1.000
15710,If God Is Willing and da Creek Don't Rise,10.000,1.000
22396,Meat the Truth,10.000,1.000
22395,Marvin Hamlisch: What He Did For Love,10.000,1.000
35343,Elaine Stritch: At Liberty,10.000,1.000


### Top Rated Threshold System

In [21]:
top_df = df.sort_values(by='vote_average', ascending=False)
top_df = top_df[top_df.vote_count > 1000]
top_df[['title','vote_average','vote_count']].head(5)

,title,vote_average,vote_count
314,The Shawshank Redemption,8.500,8358.000
834,The Godfather,8.500,6024.000
40251,Your Name.,8.500,1030.000
2843,Fight Club,8.300,9678.000
1178,The Godfather: Part II,8.300,3418.000


### Content Based Filtering

If some user likes some item, then they will like similiar systems.  Lets filter the recommendation by Genre. 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
df_descriptions = df[['title','overview']]
df_descriptions.head()

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [34]:
index = 0
values = []
keys = {}

for title_overview in df_descriptions.itertuples():
    key = title_overview[1]
    value = title_overview[2]
    values.append(value)
    keys[key] = index
    index += 1
    
    


In [35]:
vectorizer = TfidfVectorizer(stop_words = "english")

In [36]:
#Fit the vectorizer to the data
vectorizer.fit(df_descriptions['overview'].fillna(''))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [37]:
#Transform the data
tfidf_scores=vectorizer.transform(df_descriptions['overview'].fillna(''))

In [38]:
tfidf_scores

<45466x75827 sparse matrix of type '<class 'numpy.float64'>'
	with 1210882 stored elements in Compressed Sparse Row format>

In [39]:
print(tfidf_scores.shape)
print(df_descriptions.shape)

(45466, 75827)
(45466, 2)


In [43]:
pd.DataFrame(tfidf_scores.todense()).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,...,75327,75328,75329,75330,75331,75332,75333,75334,75335,75336,75337,75338,75339,75340,75341,75342,75343,75344,75345,75346,75347,75348,75349,75350,75351,75352,75353,75354,75355,75356,75357,75358,75359,75360,75361,75362,75363,75364,75365,75366,75367,75368,75369,75370,75371,75372,75373,75374,75375,75376,75377,75378,75379,75380,75381,75382,75383,75384,75385,75386,75387,75388,75389,75390,75391,75392,75393,75394,75395,75396,75397,75398,75399,75400,75401,75402,75403,75404,75405,75406,75407,75408,75409,75410,75411,75412,75413,75414,75415,75416,75417,75418,75419,75420,75421,75422,75423,75424,75425,75426,75427,75428,75429,75430,75431,75432,75433,75434,75435,75436,75437,75438,75439,75440,75441,75442,75443,75444,75445,75446,75447,75448,75449,75450,75451,75452,75453,75454,75455,75456,75457,75458,75459,75460,75461,75462,75463,75464,75465,75466,75467,75468,75469,75470,75471,75472,75473,75474,75475,75476,75477,75478,75479,75480,75481,75482,75483,75484,75485,75486,75487,75488,75489,75490,75491,75492,75493,75494,75495,75496,75497,75498,75499,75500,75501,75502,75503,75504,75505,75506,75507,75508,75509,75510,75511,75512,75513,75514,75515,75516,75517,75518,75519,75520,75521,75522,75523,75524,75525,75526,75527,75528,75529,75530,75531,75532,75533,75534,75535,75536,75537,75538,75539,75540,75541,75542,75543,75544,75545,75546,75547,75548,75549,75550,75551,75552,75553,75554,75555,75556,75557,75558,75559,75560,75561,75562,75563,75564,75565,75566,75567,75568,75569,75570,75571,75572,75573,75574,75575,75576,75577,75578,75579,75580,75581,75582,75583,75584,75585,75586,75587,75588,75589,75590,75591,75592,75593,75594,75595,75596,75597,75598,75599,75600,75601,75602,75603,75604,75605,75606,75607,75608,75609,75610,75611,75612,75613,75614,75615,75616,75617,75618,75619,75620,75621,75622,75623,75624,75625,75626,75627,75628,75629,75630,75631,75632,75633,75634,75635,75636,75637,75638,75639,75640,75641,75642,75643,75644,75645,75646,75647,75648,75649,75650,75651,75652,75653,75654,75655,75656,75657,75658,75659,75660,75661,75662,75663,75664,75665,75666,75667,75668,75669,75670,75671,75672,75673,75674,75675,75676,75677

In [45]:
#Need this helper function to look up a movie TFIDF by its name.
def get_movie_by_name(name, tfidf_scores, keys):
    rowid = keys[name]
    row = tfidf_scores[rowid,:]
    return row

In [41]:
print(tfidf_scores.shape)
print(df_descriptions.shape)

(45466, 75827)
(45466, 2)


In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

movie_1 = get_movie_by_name('No Country for Old Men',tfidf_scores,keys)
movie_2 = get_movie_by_name('The Social Network',tfidf_scores,keys)

print("Similarity:", cosine_similarity(movie_1, movie_2))

Similarity: [[0.01620919]]


In [48]:
def content_recomender(title,tfidf_scores,movies) :
    
    #Store the results in this DF
    similar_movies = pd.DataFrame(columns = ["title","similarity"] )
    
    #The movie we are finding movies similar to
    movie_1 = get_movie_by_name(title,tfidf_scores,keys)
    
    #Go through ALL the movies
    for movie in movies['title'] :
                
        #Find the similarity of the two movies
        movie_2 = get_movie_by_name(movie,tfidf_scores,keys)
        similarity = cosine_similarity(movie_1,movie_2)
        similar_movies.loc[len(similar_movies)] = [movie,similarity[0][0]]

    return similar_movies.sort_values(by=['similarity'],ascending=False)[1:]


In [50]:
similar_movies = content_recomender("Thor: Ragnarok", tfidf_scores, df.loc[df['vote_count']>1000])
similar_movies.head(10)

,title,similarity
841,Thor: The Dark World,0.245
355,The Chronicles of Riddick,0.099
467,Fantastic 4: Rise of the Silver Surfer,0.093
728,Prometheus,0.085
668,X-Men: First Class,0.082
949,Warcraft,0.082
670,Green Lantern,0.081
904,Jupiter Ascending,0.078
259,The Fast and the Furious,0.069
292,Men in Black II,0.064


### General Notes

- Top Rated Systems
- Thresholding: Top Rated with a thresholder parameter
- Content Based: Genre/Subject/History specific
- Collaboratived Based Filtering: Peer Gourp, Similiar Users in Groups
    - User Item Filtering
    - Item-Items Filtering
    - Simliar Items
    - Memory Based Filtering
- Hybrid Methods

Author: Kavi Sekhon